In [2]:
import pandas as pd
import numpy as np
import scipy as sc
import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn import linear_model
from sklearn import mixture

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

import os
import sys
import math

sys._enablelegacywindowsfsencoding()

init_notebook_mode(connected=True)

## Подготовка данных

In [159]:
df = pd.read_csv('output.csv', sep=';', encoding='utf-8')

df = df.drop(df[df.vertical_force_mean > 250].index)
df = df.drop(df[df.edge_external_force_mean > 250].index)
df = df.drop(df[df.edge_internal_force_mean > 250].index)
df = df.drop(df[df.lateral_plus_force_mean > 250].index)
df = df.drop(df[df.lateral_minus_force_mean < -250].index)

df = df.drop(df.columns[0], axis=1)

df = pd.get_dummies(df, columns=['season'])
df = pd.get_dummies(df, columns=['weight'])
df = pd.get_dummies(df, columns=['type_area'])
df = pd.get_dummies(df, columns=['type_clamping'])

df = df.rename(index=str, columns={'season_Весна': 'spring', 'season_Лето': 'summer', 'season_Осень': 'fall', 'weight_23.5': '23.5', 'weight_27.0': '27', 'type_area_Р-350': '350', 'type_area_Р-650': '650', 'type_area_прямая': '0', 'type_clamping_АРС': 'ars', 'type_clamping_ДО': 'du', 'type_clamping_КБ': 'kb'})

df.vertical_force_mean = df.vertical_force_mean * 1000
df.edge_external_force_mean = df.edge_external_force_mean * 1000
df.edge_internal_force_mean = df.edge_internal_force_mean * 1000
df.lateral_minus_force_mean = df.lateral_minus_force_mean * 1000
df.lateral_plus_force_mean = df.lateral_plus_force_mean * 1000
df.speed = df.speed * 1000 / 3600

temp = pd.read_csv('Weather.txt', sep=':', encoding='utf-8')
temp = temp.replace(' ','',regex=True)
df = pd.merge(df, temp, left_on='date', right_on='date ', how='outer')
df = df.drop('date ', axis=1)


df.temperature = df.temperature.astype('int32')

In [161]:
df_ars = df.loc[df.ars == 1]
df_du = df.loc[df.du == 1]
df_kb = df.loc[df.kb == 1]

df_ars_23_0 = df.loc[(df.ars == 1) & (df['23.5'] == 1) & (df['0'] == 1)]
df_ars_23_350 = df.loc[(df.ars == 1) & (df['23.5'] == 1) & (df['350'] == 1)]
df_ars_23_650 = df.loc[(df.ars == 1) & (df['23.5'] == 1) & (df['650'] == 1)]
df_ars_27_0 = df.loc[(df.ars == 1) & (df['27'] == 1) & (df['0'] == 1)]
df_ars_27_350 = df.loc[(df.ars == 1) & (df['27'] == 1) & (df['350'] == 1)]
df_ars_27_650 = df.loc[(df.ars == 1) & (df['27'] == 1) & (df['650'] == 1)]

df_du_23_0 = df.loc[(df.du == 1) & (df['23.5'] == 1) & (df['0'] == 1)]
df_du_23_350 = df.loc[(df.du == 1) & (df['23.5'] == 1) & (df['350'] == 1)]
df_du_23_650 = df.loc[(df.du == 1) & (df['23.5'] == 1) & (df['650'] == 1)]
df_du_27_0 = df.loc[(df.du == 1) & (df['27'] == 1) & (df['0'] == 1)]
df_du_27_350 = df.loc[(df.du == 1) & (df['27'] == 1) & (df['350'] == 1)]
df_du_27_650 = df.loc[(df.du == 1) & (df['27'] == 1) & (df['650'] == 1)]

df_kb_23_0 = df.loc[(df.kb == 1) & (df['23.5'] == 1) & (df['0'] == 1)]
df_kb_23_350 = df.loc[(df.kb == 1) & (df['23.5'] == 1) & (df['350'] == 1)]
df_kb_23_650 = df.loc[(df.kb == 1) & (df['23.5'] == 1) & (df['650'] == 1)]
df_kb_27_0 = df.loc[(df.kb == 1) & (df['27'] == 1) & (df['0'] == 1)]
df_kb_27_350 = df.loc[(df.kb == 1) & (df['27'] == 1) & (df['350'] == 1)]
df_kb_27_650 = df.loc[(df.kb == 1) & (df['27'] == 1) & (df['650'] == 1)]

## Корелляционный анализ

Для АРС

Генральная совокупность

In [102]:
df_ars.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,0.134469,-0.446335,0.193574,0.208712,-0.252434,-0.617052,0.328868,0.276202,-0.415515,0.415515,NaN,0.297019,-0.297019,NaN,NaN,NaN,0.483715
edge_internal_force_mean,0.134469,1.000000,-0.376057,-0.508033,0.136778,-0.251631,-0.371186,0.420155,-0.056177,-0.243309,0.243309,NaN,-0.733092,0.733092,NaN,NaN,NaN,0.444378
lateral_minus_force_mean,-0.446335,-0.376057,1.000000,0.373496,0.102222,0.273115,0.425038,-0.208685,-0.208099,0.298268,-0.298268,NaN,0.240147,-0.240147,NaN,NaN,NaN,-0.318937
lateral_plus_force_mean,0.193574,-0.508033,0.373496,1.000000,0.078818,0.123247,0.062063,-0.271521,0.210663,-0.148874,0.148874,NaN,0.580689,-0.580689,NaN,NaN,NaN,-0.223650
speed,0.208712,0.136778,0.102222,0.078818,1.000000,-0.021798,0.029920,-0.034033,0.004694,0.056259,-0.056259,NaN,-0.101508,0.101508,NaN,NaN,NaN,-0.022298
vertical_force_mean,-0.252434,-0.251631,0.273115,0.123247,-0.021798,1.000000,0.805629,-0.136931,-0.653054,-0.375518,0.375518,NaN,0.111243,-0.111243,NaN,NaN,NaN,-0.558510
spring,-0.617052,-0.371186,0.425038,0.062063,0.029920,0.805629,1.000000,-0.490290,-0.490290,0.053093,-0.053093,NaN,-0.038663,0.038663,NaN,NaN,NaN,-0.823195
summer,0.328868,0.420155,-0.208685,-0.271521,-0.034033,-0.136931,-0.490290,1.000000,-0.519231,-0.106153,0.106153,NaN,-0.034527,0.034527,NaN,NaN,NaN,0.839498
fall,0.276202,-0.056177,-0.208099,0.210663,0.004694,-0.653054,-0.490290,-0.519231,1.000000,0.054091,-0.054091,NaN,0.072439,-0.072439,NaN,NaN,NaN,-0.032288
23.5,-0.415515,-0.243309,0.298268,-0.148874,0.056259,-0.375518,0.053093,-0.106153,0.054091,1.000000,-1.000000,NaN,0.010283,-0.010283,NaN,NaN,NaN,-0.015326


In [103]:
df_ars_23_0.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,0.784962,-0.318797,0.203008,0.227258,-0.496241,-0.756876,0.627195,0.099988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.749283
edge_internal_force_mean,0.784962,1.000000,-0.251128,0.124812,0.186487,-0.487218,-0.681188,0.481758,0.172706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.637574
lateral_minus_force_mean,-0.318797,-0.251128,1.000000,0.338346,-0.011325,0.136842,0.302750,-0.154526,-0.136347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.195300
lateral_plus_force_mean,0.203008,0.124812,0.338346,1.000000,0.348059,-0.127820,0.113531,-0.245424,0.136347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.253814
speed,0.227258,0.186487,-0.011325,0.348059,1.000000,-0.037750,0.133004,-0.118659,-0.009128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.079742
vertical_force_mean,-0.496241,-0.487218,0.136842,-0.127820,-0.037750,1.000000,0.794719,0.063628,-0.827170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.357163
spring,-0.756876,-0.681188,0.302750,0.113531,0.133004,0.794719,1.000000,-0.480384,-0.480384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.803219
summer,0.627195,0.481758,-0.154526,-0.245424,-0.118659,0.063628,-0.480384,1.000000,-0.538462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.836017
fall,0.099988,0.172706,-0.136347,0.136347,-0.009128,-0.827170,-0.480384,-0.538462,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.064309
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
df_ars_23_650.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,0.217750,-0.300877,0.601651,0.553433,-0.254902,-0.406364,-0.051514,0.450296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.245444
edge_internal_force_mean,0.217750,1.000000,-0.817578,0.027864,-0.028911,-0.459236,-0.779780,0.721196,0.164742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.913305
lateral_minus_force_mean,-0.300877,-0.817578,1.000000,-0.056348,0.169131,0.677239,0.871227,-0.557239,-0.396012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.858420
lateral_plus_force_mean,0.601651,0.027864,-0.056348,1.000000,0.159009,-0.135191,-0.208674,-0.180299,0.362433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.066365
speed,0.553433,-0.028911,0.169131,0.159009,1.000000,0.206505,0.021977,-0.025770,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000527
vertical_force_mean,-0.254902,-0.459236,0.677239,-0.135191,0.206505,1.000000,0.845677,0.000000,-0.845677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.537238
spring,-0.406364,-0.779780,0.871227,-0.208674,0.021977,0.845677,1.000000,-0.426401,-0.636364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.863227
summer,-0.051514,0.721196,-0.557239,-0.180299,-0.025770,0.000000,-0.426401,1.000000,-0.426401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.736163
fall,0.450296,0.164742,-0.396012,0.362433,0.000000,-0.845677,-0.636364,-0.426401,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.235426
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
df_ars_27_0.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,0.783173,-0.268210,-0.106719,0.386829,-0.722191,-0.774482,0.402135,0.402181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.651856
edge_internal_force_mean,0.783173,1.000000,0.049125,-0.059289,0.286287,-0.512140,-0.834058,0.834058,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.916217
lateral_minus_force_mean,-0.268210,0.049125,1.000000,0.583286,0.065892,0.323546,0.059576,0.208514,-0.289570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.067695
lateral_plus_force_mean,-0.106719,-0.059289,0.583286,1.000000,0.159617,0.123659,0.014894,0.000000,-0.016087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.040850
speed,0.386829,0.286287,0.065892,0.159617,1.000000,-0.095997,-0.097389,0.142338,-0.048550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.099215
vertical_force_mean,-0.722191,-0.512140,0.323546,0.123659,-0.095997,1.000000,0.834058,-0.119151,-0.772187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.536891
spring,-0.774482,-0.834058,0.059576,0.014894,-0.097389,0.834058,1.000000,-0.571429,-0.462910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.862012
summer,0.402135,0.834058,0.208514,0.000000,0.142338,-0.119151,-0.571429,1.000000,-0.462910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.862012
fall,0.402181,0.000000,-0.289570,-0.016087,-0.048550,-0.772187,-0.462910,-0.462910,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
df_ars_27_650.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,0.256140,-0.391193,0.150877,0.362039,-0.484211,-0.707107,0.194625,0.398410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.452597
edge_internal_force_mean,0.256140,1.000000,-0.352426,-0.617544,-0.244597,0.056140,-0.565685,0.856349,-0.398410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.874272
lateral_minus_force_mean,-0.391193,-0.352426,1.000000,0.037886,0.227938,0.512780,0.710229,-0.390968,-0.200084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.592951
lateral_plus_force_mean,0.150877,-0.617544,0.037886,1.000000,0.368220,-0.122807,0.164992,-0.544949,0.418330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.454471
speed,0.362039,-0.244597,0.227938,0.368220,1.000000,-0.004415,0.142361,-0.195918,0.080211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.198089
vertical_force_mean,-0.484211,0.056140,0.512780,-0.122807,-0.004415,1.000000,0.707107,0.233550,-0.836660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.134936
spring,-0.707107,-0.565685,0.710229,0.164992,0.142361,0.707107,1.000000,-0.440386,-0.394405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.755361
summer,0.194625,0.856349,-0.390968,-0.544949,-0.195918,0.233550,-0.440386,1.000000,-0.651339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.914788
fall,0.398410,-0.398410,-0.200084,0.418330,0.080211,-0.836660,-0.394405,-0.651339,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.297918
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Для ДУ

In [107]:
df_du.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,0.527065,-0.266816,-0.025408,0.186377,-0.095542,-0.132025,0.031743,0.093660,-0.237023,0.237023,-0.569984,0.736663,-0.171505,NaN,NaN,NaN,0.099170
edge_internal_force_mean,0.527065,1.000000,-0.440432,-0.131650,0.208044,0.145577,0.126284,-0.251159,0.125061,-0.041951,0.041951,-0.796695,0.375168,0.490712,NaN,NaN,NaN,-0.225903
lateral_minus_force_mean,-0.266816,-0.440432,1.000000,0.161537,-0.099531,-0.078779,0.287317,-0.360575,0.079552,-0.004356,0.004356,0.534434,-0.205872,-0.380285,NaN,NaN,NaN,-0.352152
lateral_plus_force_mean,-0.025408,-0.131650,0.161537,1.000000,0.029598,0.323057,0.032145,0.187673,-0.212766,-0.264816,0.264816,0.458768,-0.315010,-0.172115,NaN,NaN,NaN,0.108093
speed,0.186377,0.208044,-0.099531,0.029598,1.000000,0.024296,0.282562,-0.023959,-0.243208,0.073189,-0.073189,-0.190449,0.139255,0.061981,NaN,NaN,NaN,-0.232579
vertical_force_mean,-0.095542,0.145577,-0.078779,0.323057,0.024296,1.000000,0.102176,0.149247,-0.241459,-0.539070,0.539070,0.011763,-0.361495,0.390006,NaN,NaN,NaN,0.052998
spring,-0.132025,0.126284,0.287317,0.032145,0.282562,0.102176,1.000000,-0.455138,-0.500685,0.066342,-0.066342,-0.062235,-0.004843,0.076444,NaN,NaN,NaN,-0.808161
summer,0.031743,-0.251159,-0.360575,0.187673,-0.023959,0.149247,-0.455138,1.000000,-0.542893,-0.154471,0.154471,0.105661,0.094646,-0.226236,NaN,NaN,NaN,0.833015
fall,0.093660,0.125061,0.079552,-0.212766,-0.243208,-0.241459,-0.500685,-0.542893,1.000000,0.087599,-0.087599,-0.044023,-0.087443,0.147838,NaN,NaN,NaN,-0.047604
23.5,-0.237023,-0.041951,-0.004356,-0.264816,0.073189,-0.539070,0.066342,-0.154471,0.087599,1.000000,-1.000000,-0.035533,0.014222,0.024689,NaN,NaN,NaN,-0.095435


In [108]:
df_du_23_0.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,0.797802,-0.221978,0.252747,0.136414,-0.037363,-0.129430,-0.447214,0.358057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
edge_internal_force_mean,0.797802,1.000000,-0.028571,0.248352,0.037404,-0.323077,-0.462250,-0.378412,0.644503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277108
lateral_minus_force_mean,-0.221978,-0.028571,1.000000,-0.094505,-0.365237,-0.235165,-0.314330,-0.240807,0.429669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.229409
lateral_plus_force_mean,0.252747,0.248352,-0.094505,1.000000,0.101210,0.213187,-0.203390,0.447214,-0.035806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.411120
speed,0.136414,0.037404,-0.365237,0.101210,1.000000,0.605061,0.462759,0.103317,-0.501832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.441132
vertical_force_mean,-0.037363,-0.323077,-0.235165,0.213187,0.605061,1.000000,0.684130,0.240807,-0.787726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.436105
spring,-0.129430,-0.462250,-0.314330,-0.203390,0.462759,0.684130,1.000000,-0.206725,-0.860663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.859905
summer,-0.447214,-0.378412,-0.240807,0.447214,0.103317,0.240807,-0.206725,1.000000,-0.320256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.462185
fall,0.358057,0.644503,0.429669,-0.035806,-0.501832,-0.787726,-0.860663,-0.320256,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.592071
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
df_du_23_650.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,-0.466165,-0.255051,-0.278195,-0.532581,-0.547368,-0.481758,-0.151375,0.627195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260639
edge_internal_force_mean,-0.466165,1.000000,-0.036875,0.634586,0.592091,0.697744,0.409040,0.435203,-0.827170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.030573
lateral_minus_force_mean,-0.255051,-0.036875,1.000000,-0.275025,0.000770,0.439426,0.845154,-0.811998,-0.065012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.947298
lateral_plus_force_mean,-0.278195,0.634586,-0.275025,1.000000,0.288513,0.545865,0.172706,0.681188,-0.827170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.338601
speed,-0.532581,0.592091,0.000770,0.288513,1.000000,0.332957,0.209459,0.303321,-0.500880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.063560
vertical_force_mean,-0.547368,0.697744,0.439426,0.545865,0.332957,1.000000,0.808991,0.018922,-0.827170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.449430
spring,-0.481758,0.409040,0.845154,0.172706,0.209459,0.808991,1.000000,-0.480384,-0.538462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.840875
summer,-0.151375,0.435203,-0.811998,0.681188,0.303321,0.018922,-0.480384,1.000000,-0.480384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.807887
fall,0.627195,-0.827170,-0.065012,-0.827170,-0.500880,-0.827170,-0.538462,-0.480384,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.064683
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
df_du_27_0.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,0.446154,-0.342243,0.323077,0.327473,0.182418,-0.647150,0.280692,0.393863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.540118
edge_internal_force_mean,0.446154,1.000000,0.317797,0.556044,-0.169231,0.072527,-0.351310,-0.626159,0.859338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.067225
lateral_minus_force_mean,-0.342243,0.317797,1.000000,-0.253349,-0.404469,-0.073338,-0.243056,-0.414826,0.579293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007032
lateral_plus_force_mean,0.323077,0.556044,-0.253349,1.000000,0.125275,0.054945,0.129430,-0.367058,0.179029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.340761
speed,0.327473,-0.169231,-0.404469,0.125275,1.000000,-0.186813,0.092450,0.194325,-0.250640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.076497
vertical_force_mean,0.182418,0.072527,-0.073338,0.054945,-0.186813,1.000000,-0.018490,0.064775,-0.035806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.134450
spring,-0.647150,-0.351310,-0.243056,0.129430,0.092450,-0.018490,1.000000,-0.389249,-0.645497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.877594
summer,0.280692,-0.626159,-0.414826,-0.367058,0.194325,0.064775,-0.389249,1.000000,-0.452267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.751527
fall,0.393863,0.859338,0.579293,0.179029,-0.250640,-0.035806,-0.645497,-0.452267,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.226594
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
df_du_27_650.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,0.081818,-0.647005,0.509091,-0.326581,-0.188312,-0.683158,0.645583,-0.069631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.801618
edge_internal_force_mean,0.081818,1.000000,-0.449948,0.428571,0.442422,0.677922,0.092319,0.440886,-0.574456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.245421
lateral_minus_force_mean,-0.647005,-0.449948,1.000000,-0.648319,-0.210859,-0.042696,0.700394,-0.876038,0.308158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.895550
lateral_plus_force_mean,0.509091,0.428571,-0.648319,1.000000,-0.060211,0.371429,-0.203101,0.850279,-0.748534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.651565
speed,-0.326581,0.442422,-0.210859,-0.060211,1.000000,0.462056,0.158179,0.071415,-0.228085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.061167
vertical_force_mean,-0.188312,0.677922,-0.042696,0.371429,0.462056,1.000000,0.609303,0.188951,-0.783349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.118042
spring,-0.683158,0.092319,0.700394,-0.203101,0.158179,0.609303,1.000000,-0.533002,-0.353553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.758513
summer,0.645583,0.440886,-0.876038,0.850279,0.071415,0.188951,-0.533002,1.000000,-0.603023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.889436
fall,-0.069631,-0.574456,0.308158,-0.748534,-0.228085,-0.783349,-0.353553,-0.603023,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.268175
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Для КБ

In [116]:
df_kb.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,-0.497479,0.055726,0.695556,-0.108014,-0.193734,-0.354139,0.225694,0.146608,-0.177084,0.177084,0.660634,0.165810,-0.793586,NaN,NaN,NaN,0.254197
edge_internal_force_mean,-0.497479,1.000000,-0.002433,-0.342656,0.166712,0.242449,0.241255,-0.396004,0.180875,-0.150399,0.150399,-0.340953,-0.252375,0.567733,NaN,NaN,NaN,-0.319692
lateral_minus_force_mean,0.055726,-0.002433,1.000000,0.041190,-0.203687,0.510321,0.505966,-0.305683,-0.226358,-0.210341,0.210341,0.030860,0.092170,-0.118000,NaN,NaN,NaN,-0.256333
lateral_plus_force_mean,0.695556,-0.342656,0.041190,1.000000,0.019193,-0.008501,-0.100627,0.088959,0.012513,-0.119964,0.119964,0.672268,-0.054819,-0.588603,NaN,NaN,NaN,0.120264
speed,-0.108014,0.166712,-0.203687,0.019193,1.000000,0.054746,-0.064636,0.191825,-0.147398,-0.099729,0.099729,-0.155847,0.050969,0.099688,NaN,NaN,NaN,0.246893
vertical_force_mean,-0.193734,0.242449,0.510321,-0.008501,0.054746,1.000000,0.784893,-0.358550,-0.484612,-0.386271,0.386271,-0.107053,0.115328,-0.008696,NaN,NaN,NaN,-0.236653
spring,-0.354139,0.241255,0.505966,-0.100627,-0.064636,0.784893,1.000000,-0.621531,-0.427327,0.058978,-0.058978,-0.069889,-0.009251,0.075556,NaN,NaN,NaN,-0.535404
summer,0.225694,-0.396004,-0.305683,0.088959,0.191825,-0.358550,-0.621531,1.000000,-0.442662,-0.002100,0.002100,0.018792,0.077683,-0.092569,NaN,NaN,NaN,0.876343
fall,0.146608,0.180875,-0.226358,0.012513,-0.147398,-0.484612,-0.427327,-0.442662,1.000000,-0.065085,0.065085,0.058308,-0.079064,0.020349,NaN,NaN,NaN,-0.398536
23.5,-0.177084,-0.150399,-0.210341,-0.119964,-0.099729,-0.386271,0.058978,-0.002100,-0.065085,1.000000,-1.000000,-0.024414,-0.074916,0.095273,NaN,NaN,NaN,-0.129337


In [112]:
df_kb_23_0.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,-0.415893,-0.077560,-0.155831,0.358286,-0.567889,-0.861994,0.657454,0.298843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.579955
edge_internal_force_mean,-0.415893,1.000000,-0.061014,0.275542,-0.076407,0.422303,0.215499,-0.657454,0.418380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.383794
lateral_minus_force_mean,-0.077560,-0.061014,1.000000,0.114788,-0.491465,-0.205898,0.172756,-0.275505,0.083849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.519194
lateral_plus_force_mean,-0.155831,0.275542,0.114788,1.000000,0.201342,0.214765,0.064650,-0.370565,0.298843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.257995
speed,0.358286,-0.076407,-0.491465,0.201342,1.000000,0.014463,-0.344976,0.322917,0.059799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.595187
vertical_force_mean,-0.567889,0.422303,-0.205898,0.214765,0.014463,1.000000,0.646830,-0.179398,-0.538195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001067
spring,-0.861994,0.215499,0.172756,0.064650,-0.344976,0.646830,1.000000,-0.554700,-0.554700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.434108
summer,0.657454,-0.657454,-0.275505,-0.370565,0.322917,-0.179398,-0.554700,1.000000,-0.384615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.802666
fall,0.298843,0.418380,0.083849,0.298843,0.059799,-0.538195,-0.554700,-0.384615,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.321067
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df_kb_23_650.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,-0.545455,-0.023262,0.454545,-0.163636,0.381818,0.059761,0.173205,-0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.273842
edge_internal_force_mean,-0.545455,1.000000,-0.362889,-0.081818,0.318182,0.009091,-0.119523,0.173205,-0.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.162448
lateral_minus_force_mean,-0.023262,-0.362889,1.000000,0.348932,-0.562943,0.530376,0.856349,-0.886405,0.102353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.665085
lateral_plus_force_mean,0.454545,-0.081818,0.348932,1.000000,-0.645455,0.600000,0.418330,-0.288675,-0.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.060338
speed,-0.163636,0.318182,-0.562943,-0.645455,1.000000,-0.154545,-0.298807,0.519615,-0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.273842
vertical_force_mean,0.381818,0.009091,0.530376,0.600000,-0.154545,1.000000,0.836660,-0.519615,-0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.301690
spring,0.059761,-0.119523,0.856349,0.418330,-0.298807,0.836660,1.000000,-0.828079,-0.239046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.610227
summer,0.173205,0.173205,-0.886405,-0.288675,0.519615,-0.519615,-0.828079,1.000000,-0.346410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.884302
fall,-0.400000,-0.100000,0.102353,-0.200000,-0.400000,-0.500000,-0.239046,-0.346410,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.510552
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
df_kb_27_0.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,0.233504,0.460723,0.020171,-0.002396,-0.290256,-0.399629,0.115950,0.357830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.085893
edge_internal_force_mean,0.233504,1.000000,0.099204,0.291819,-0.014672,0.145910,0.251618,-0.580968,0.377445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.652080
lateral_minus_force_mean,0.460723,0.099204,1.000000,-0.157380,-0.345240,-0.047765,-0.135902,-0.009877,0.172092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.085860
lateral_plus_force_mean,0.020171,0.291819,-0.157380,1.000000,0.454226,0.202076,0.106454,-0.039388,-0.080064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.014427
speed,-0.002396,-0.014672,-0.345240,0.454226,1.000000,-0.104234,-0.067837,0.108465,-0.045814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.070787
vertical_force_mean,-0.290256,0.145910,-0.047765,0.202076,-0.104234,1.000000,0.851631,-0.246173,-0.720577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.250060
spring,-0.399629,0.251618,-0.135902,0.106454,-0.067837,0.851631,1.000000,-0.635934,-0.443203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.589509
summer,0.115950,-0.580968,-0.009877,-0.039388,0.108465,-0.246173,-0.635934,1.000000,-0.409960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.879049
fall,0.357830,0.377445,0.172092,-0.080064,-0.045814,-0.720577,-0.443203,-0.409960,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.324337
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
df_kb_27_650.corr('spearman')

,edge_external_force_mean,edge_internal_force_mean,lateral_minus_force_mean,lateral_plus_force_mean,speed,vertical_force_mean,spring,summer,fall,23.5,27,350,650,0,ars,du,kb,temperature
edge_external_force_mean,1.000000,-0.818681,-0.279385,0.473138,0.083092,-0.346764,-0.580968,0.745177,-0.205879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.719465
edge_internal_force_mean,-0.818681,1.000000,0.358397,-0.271673,0.068734,0.547619,0.679438,-0.754854,0.102940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.749054
lateral_minus_force_mean,-0.279385,0.358397,1.000000,-0.025916,-0.011386,0.705416,0.815616,-0.170338,-0.746063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.184813
lateral_plus_force_mean,0.473138,-0.271673,-0.025916,1.000000,0.224225,-0.064713,-0.128010,0.164520,-0.045751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.137653
speed,0.083092,0.068734,-0.011386,0.224225,1.000000,0.155797,-0.078836,0.246968,-0.200313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.283886
vertical_force_mean,-0.346764,0.547619,0.705416,-0.064713,0.155797,1.000000,0.836988,-0.464526,-0.423196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.430328
spring,-0.580968,0.679438,0.815616,-0.128010,-0.078836,0.836988,1.000000,-0.635934,-0.409960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.591375
summer,0.745177,-0.754854,-0.170338,0.164520,0.246968,-0.464526,-0.635934,1.000000,-0.443203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.897302
fall,-0.205879,0.102940,-0.746063,-0.045751,-0.200313,-0.423196,-0.409960,-0.443203,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.373584
23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Статистические критерии

Проверим, как на силы влияет тип скрепления. Используем критерий Краскела — Уоллиса

https://en.wikipedia.org/wiki/Kruskal–Wallis_one-way_analysis_of_variance

Масса 23.5 тонн/ось, кривизна 0, лето

In [78]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].vertical_force_mean,
                df_du_23_0.loc[df_du_23_0.summer == 1].vertical_force_mean,
                df_kb_23_0.loc[df_kb_23_0.summer == 1].vertical_force_mean)

KruskalResult(statistic=3.0375442739079057, pvalue=0.21898060013366696)

In [118]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].lateral_plus_force_mean,
                df_du_23_0.loc[df_du_23_0.summer == 1].lateral_plus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.summer == 1].lateral_plus_force_mean)

KruskalResult(statistic=4.008791208791209, pvalue=0.13474170838656888)

In [119]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].lateral_minus_force_mean,
                df_du_23_0.loc[df_du_23_0.summer == 1].lateral_minus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.summer == 1].lateral_minus_force_mean)

KruskalResult(statistic=0.33532182103611063, pvalue=0.8456405338841163)

In [120]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].edge_internal_force_mean,
                df_du_23_0.loc[df_du_23_0.summer == 1].edge_internal_force_mean,
                df_kb_23_0.loc[df_kb_23_0.summer == 1].edge_internal_force_mean)

KruskalResult(statistic=4.611616954474094, pvalue=0.09967818052825451)

In [121]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].edge_external_force_mean,
                df_du_23_0.loc[df_du_23_0.summer == 1].edge_external_force_mean,
                df_kb_23_0.loc[df_kb_23_0.summer == 1].edge_external_force_mean)

KruskalResult(statistic=6.103610675039249, pvalue=0.04727350267876055)

Масса 23.5 тонн/ось, кривизна 0, весна

In [79]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.spring == 1].vertical_force_mean,
                df_du_23_0.loc[df_du_23_0.spring == 1].vertical_force_mean,
                df_kb_23_0.loc[df_kb_23_0.spring == 1].vertical_force_mean)

KruskalResult(statistic=8.377894736842109, pvalue=0.015162236725046708)

In [122]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.spring == 1].lateral_plus_force_mean,
                df_du_23_0.loc[df_du_23_0.spring == 1].lateral_plus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.spring == 1].lateral_plus_force_mean)

KruskalResult(statistic=1.2463157894736838, pvalue=0.5362483451366988)

In [123]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.spring == 1].lateral_minus_force_mean,
                df_du_23_0.loc[df_du_23_0.spring == 1].lateral_minus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.spring == 1].lateral_minus_force_mean)

KruskalResult(statistic=0.10985915492957887, pvalue=0.9465518041841647)

In [124]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.spring == 1].edge_internal_force_mean,
                df_du_23_0.loc[df_du_23_0.spring == 1].edge_internal_force_mean,
                df_kb_23_0.loc[df_kb_23_0.spring == 1].edge_internal_force_mean)

KruskalResult(statistic=3.7252631578947373, pvalue=0.15526350411136827)

In [125]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.spring == 1].edge_external_force_mean,
                df_du_23_0.loc[df_du_23_0.spring == 1].edge_external_force_mean,
                df_kb_23_0.loc[df_kb_23_0.spring == 1].edge_external_force_mean)

KruskalResult(statistic=13.3042105263158, pvalue=0.0012913007140293988)

Масса 23.5 тонн/ось, кривизна 0, осень

In [80]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.fall == 1].vertical_force_mean,
                df_du_23_0.loc[df_du_23_0.fall == 1].vertical_force_mean,
                df_kb_23_0.loc[df_kb_23_0.fall == 1].vertical_force_mean)

KruskalResult(statistic=12.622448979591837, pvalue=0.0018158084384611104)

In [126]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.fall == 1].lateral_plus_force_mean,
                df_du_23_0.loc[df_du_23_0.fall == 1].lateral_plus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.fall == 1].lateral_plus_force_mean)

KruskalResult(statistic=2.263163265306119, pvalue=0.322522740306774)

In [127]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.fall == 1].lateral_minus_force_mean,
                df_du_23_0.loc[df_du_23_0.fall == 1].lateral_minus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.fall == 1].lateral_minus_force_mean)

KruskalResult(statistic=4.8367346938775455, pvalue=0.08906691418764005)

In [128]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.fall == 1].edge_internal_force_mean,
                df_du_23_0.loc[df_du_23_0.fall == 1].edge_internal_force_mean,
                df_kb_23_0.loc[df_kb_23_0.fall == 1].edge_internal_force_mean)

KruskalResult(statistic=10.34693877551021, pvalue=0.005664881017821615)

In [129]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.fall == 1].edge_external_force_mean,
                df_du_23_0.loc[df_du_23_0.fall == 1].edge_external_force_mean,
                df_kb_23_0.loc[df_kb_23_0.fall == 1].edge_external_force_mean)

KruskalResult(statistic=9.857142857142861, pvalue=0.007236834234988313)

Масса 23.5 тонн/ось, кривизна 650, лето

In [84]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.summer == 1].vertical_force_mean,
                df_du_23_650.loc[df_du_23_650.summer == 1].vertical_force_mean,
                df_kb_23_650.loc[df_kb_23_650.summer == 1].vertical_force_mean)

KruskalResult(statistic=12.426470588235304, pvalue=0.002002747494063425)

Масса 23.5 тонн/ось, кривизна 650, весна

In [86]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.spring == 1].vertical_force_mean,
                df_du_23_650.loc[df_du_23_650.spring == 1].vertical_force_mean,
                df_kb_23_650.loc[df_kb_23_650.spring == 1].vertical_force_mean)

KruskalResult(statistic=14.859649122807014, pvalue=0.0005932915890212373)

Масса 23.5 тонн/ось, кривизна 650, осень

In [85]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.fall == 1].vertical_force_mean,
                df_du_23_650.loc[df_du_23_650.fall == 1].vertical_force_mean,
                df_kb_23_650.loc[df_kb_23_650.fall == 1].vertical_force_mean)

KruskalResult(statistic=11.200000000000003, pvalue=0.0036978637164829255)

Масса 27 тонн/ось, кривизна 0, лето

In [92]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.summer == 1].vertical_force_mean,
                df_du_27_0.loc[df_du_27_0.summer == 1].vertical_force_mean,
                df_kb_27_0.loc[df_kb_27_0.summer == 1].vertical_force_mean)

KruskalResult(statistic=6.077922077922082, pvalue=0.04788461390707669)

Масса 27 тонн/ось, кривизна 0, весна

In [93]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.spring == 1].vertical_force_mean,
                df_du_27_0.loc[df_du_27_0.spring == 1].vertical_force_mean,
                df_kb_27_0.loc[df_kb_27_0.spring == 1].vertical_force_mean)

KruskalResult(statistic=16.102500000000006, pvalue=0.00031870329427263614)

Масса 27 тонн/ось, кривизна 0, осень

In [94]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.fall == 1].vertical_force_mean,
                df_du_27_0.loc[df_du_27_0.fall == 1].vertical_force_mean,
                df_kb_27_0.loc[df_kb_27_0.fall == 1].vertical_force_mean)

KruskalResult(statistic=14.3625730994152, pvalue=0.0007606885487011675)

Масса 27 тонн/ось, кривизна 650, лето

In [95]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.summer == 1].vertical_force_mean,
                df_du_27_650.loc[df_du_27_650.summer == 1].vertical_force_mean,
                df_kb_27_650.loc[df_kb_27_650.summer == 1].vertical_force_mean)

KruskalResult(statistic=3.676457680250792, pvalue=0.1590989664122756)

Масса 27 тонн/ось, кривизна 650, весна

In [99]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.spring == 1].vertical_force_mean,
                df_du_27_650.loc[df_du_27_650.spring == 1].vertical_force_mean,
                df_kb_27_650.loc[df_kb_27_650.spring == 1].vertical_force_mean)

KruskalResult(statistic=14.921052631578945, pvalue=0.0005753532742717005)

Масса 27 тонн/ось, кривизна 650, осень

In [98]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.fall == 1].vertical_force_mean,
                df_du_27_650.loc[df_du_27_650.fall == 1].vertical_force_mean,
                df_kb_27_650.loc[df_kb_27_650.fall == 1].vertical_force_mean)

KruskalResult(statistic=14.866917293233087, pvalue=0.0005911394297641764)

Теперь проверим, как масса влияет на силы

## Визуализация данных для боковых сил

In [163]:
df_tmp = df_ars_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [164]:
df_tmp = df_ars_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [165]:
df_tmp = df_ars_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [166]:
df_tmp = df_ars_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [168]:
df_tmp = df_du_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [180]:
df_tmp = df_du_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [181]:
df_tmp = df_du_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [182]:
df_tmp = df_du_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [185]:
df_tmp = df_kb_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [186]:
df_tmp = df_kb_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [187]:
df_tmp = df_kb_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [188]:
df_tmp = df_kb_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

## Визуализация данных для положительных боковых сил

In [199]:
df_tmp = df_ars_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [200]:
df_tmp = df_ars_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [201]:
df_tmp = df_ars_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [202]:
df_tmp = df_ars_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [203]:
df_tmp = df_du_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [204]:
df_tmp = df_du_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [206]:
df_tmp = df_du_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [207]:
df_tmp = df_du_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [208]:
df_tmp = df_kb_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [209]:
df_tmp = df_kb_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [210]:
df_tmp = df_kb_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [211]:
df_tmp = df_kb_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

## Визуализация данных для боковых отрицательных сил

In [213]:
df_tmp = df_ars_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [214]:
df_tmp = df_ars_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [215]:
df_tmp = df_ars_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [216]:
df_tmp = df_ars_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [218]:
df_tmp = df_du_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [219]:
df_tmp = df_du_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [220]:
df_tmp = df_du_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [222]:
df_tmp = df_du_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'ДУ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [223]:
df_tmp = df_kb_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [224]:
df_tmp = df_kb_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [225]:
df_tmp = df_kb_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [226]:
df_tmp = df_kb_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'КБ',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

## Визуализация данных для внешней кромки

In [227]:
df_tmp = df_ars_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [228]:
df_tmp = df_ars_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [229]:
df_tmp = df_ars_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [231]:
df_tmp = df_du_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [232]:
df_tmp = df_du_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [233]:
df_tmp = df_du_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [234]:
df_tmp = df_du_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [235]:
df_tmp = df_kb_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [236]:
df_tmp = df_du_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [237]:
df_tmp = df_kb_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [238]:
df_tmp = df_kb_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

## Визуализация данных для внутренней кромочной силы 

In [239]:
df_tmp = df_ars_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [240]:
df_tmp = df_ars_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [241]:
df_tmp = df_ars_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [242]:
df_tmp = df_ars_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [243]:
df_tmp = df_du_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [244]:
df_tmp = df_du_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [245]:
df_tmp = df_du_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [246]:
df_tmp = df_du_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [247]:
df_tmp = df_kb_23_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [248]:
df_tmp = df_kb_23_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [249]:
df_tmp = df_kb_27_0

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

In [250]:
df_tmp = df_kb_27_650

trace_summer = go.Scatter(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)


trace_summer = go.Scatter3d(
    x=df_tmp.loc[df_tmp.summer == 1].speed,
    y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.summer == 1].temperature,
    mode='markers',
    name='Лето',
    marker = dict(
        size = 12,
        color = '#4F7942',
        line = dict(
            width = 1,
        )
    )
)

trace_fall = go.Scatter3d(
    x=df_tmp.loc[df_tmp.fall == 1].speed,
    y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.fall == 1].temperature,
    mode='markers',
    name='Осень',
    marker = dict(
        size = 12,
        color = '#FFA500',
        line = dict(
            width = 1,
        )
    )
)

trace_spring = go.Scatter3d(
    x=df_tmp.loc[df_tmp.spring == 1].speed,
    y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
    z=df_tmp.loc[df_tmp.spring == 1].temperature,
    mode='markers',
    name='Весна',
    marker = dict(
        size = 12,
        color = '#00BFFF',
        line = dict(
            width = 1,
        )
    )
)

data = [trace_summer, trace_fall, trace_spring]

layout = dict(title = 'АРС',
              yaxis = dict(title='Сила'),
              xaxis = dict(title='Скорость'))

fig = dict(data=data, layout=layout)
iplot(fig)

## Полиноминальный регрессор для вертикальных сил

In [251]:
df_list = [df_ars_23_0, df_ars_23_650, df_ars_27_0, df_ars_27_650, df_du_23_0, df_du_23_650, df_du_27_0, df_du_27_650, df_kb_23_0, df_kb_23_650, df_kb_27_0, df_kb_27_650]

In [257]:
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    ridge1 = Ridge()
    model1 = make_pipeline(PolynomialFeatures(2), ridge1)
    model1.fit(np.array(df_tmp.loc[df_tmp.summer == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.summer == 1].vertical_force_mean))

    trace_summer_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model1.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для лета',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )
    
    print('Лето: ' + str([ridge1.intercept_, ridge1.coef_[1], ridge1.coef_[2]]))

    ridge2 = Ridge()
    model2 = make_pipeline(PolynomialFeatures(2), ridge2)
    model2.fit(np.array(df_tmp.loc[df_tmp.fall == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.fall == 1].vertical_force_mean))

    trace_fall_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model2.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для осени',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Осень: ' + str([ridge2.intercept_, ridge2.coef_[1], ridge2.coef_[2]]))

    ridge3 = Ridge()
    model3 = make_pipeline(PolynomialFeatures(2), ridge3)
    model3.fit(np.array(df_tmp.loc[df_tmp.spring == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.spring == 1].vertical_force_mean))

    trace_spring_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model3.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для весны',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Весна: ' + str([ridge3.intercept_, ridge3.coef_[1], ridge3.coef_[2]]))

    trace_summer = go.Scatter(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer_predict, trace_fall_predict, trace_spring_predict, trace_summer, trace_fall, trace_spring]

    layout = dict(title = 'АРС',
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)

Лето: [125069.26787046838, -1148.4484462093108, 27.34623182652281]
Осень: [121552.31263282467, -1518.0536977169093, 37.92300716405643]
Весна: [193364.16746910158, -4809.568257250429, 154.48619839579803]


Лето: [124275.00930568905, -1400.7094109607908, 44.31743757933436]
Осень: [114868.7765741049, -1255.8302853882826, 43.38724446667949]
Весна: [142596.2970427154, 6979.647119877698, -157.7231239914525]


Лето: [146921.68942651537, -1978.0343564522477, 48.0177893157277]
Осень: [112120.25953640058, 930.8295048479908, -30.604186550556523]
Весна: [149041.5278949359, 2780.614892250742, -56.68860759119759]


Лето: [137877.5381659224, -501.1070887810837, 10.732763856485533]
Осень: [111074.87079400479, 1265.848064885907, -36.34610341732957]
Весна: [210187.23226276902, 3151.9410222906554, -89.88712972660913]


Лето: [136202.0, 0.0, 0.0]
Осень: [112228.71831844494, 1042.9649035715856, -28.982785325659986]
Весна: [115847.7646842465, 2872.752899698734, -79.78684951508234]


...

Лето: [120402.88073142982, -121.94831717461594, 48.38702900851072]
Осень: [57271.39875787368, 1465.5166166172924, -1.424553222056839]
Весна: [124285.38514390484, 151.02839638459773, 52.59537538280794]


Лето: [118905.16379476449, -356.7934411521646, 5.303429795635912]
Осень: [90700.35256855348, 2510.1060013034967, -79.51542066383178]
Весна: [91528.84944339548, 9174.242453295448, -347.2617379286489]


Лето: [108286.22785250634, 1804.0057876030928, -74.08627231415517]
Осень: [111396.0, 0.0, 0.0]
Весна: [148824.12117295898, -128.3759590631754, 41.28978801948513]


Лето: [125222.49641927356, 1027.8397438327427, -37.83797359806445]
Осень: [130439.8530121914, -699.5610248643706, 6.143680980899542]
Весна: [159380.25125231603, 1635.4699339802585, -64.76952934143264]


Лето: [131396.3680498277, -287.8035748159864, 17.813640148669357]
Осень: [139716.52861163163, -3167.444734325426, 177.6331423291667]
Весна: [141006.7495740088, 2685.109912564271, -25.473194108865908]


In [259]:
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    ridge1 = Ridge()
    model1 = make_pipeline(PolynomialFeatures(2), ridge1)
    model1.fit(np.array(df_tmp.loc[df_tmp.summer == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean))

    trace_summer_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model1.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для лета',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )
    
    print('Лето: ' + str([ridge1.intercept_, ridge1.coef_[1], ridge1.coef_[2]]))

    ridge2 = Ridge()
    model2 = make_pipeline(PolynomialFeatures(2), ridge2)
    model2.fit(np.array(df_tmp.loc[df_tmp.fall == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean))

    trace_fall_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model2.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для осени',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Осень: ' + str([ridge2.intercept_, ridge2.coef_[1], ridge2.coef_[2]]))

    ridge3 = Ridge()
    model3 = make_pipeline(PolynomialFeatures(2), ridge3)
    model3.fit(np.array(df_tmp.loc[df_tmp.spring == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean))

    trace_spring_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model3.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для весны',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Весна: ' + str([ridge3.intercept_, ridge3.coef_[1], ridge3.coef_[2]]))

    trace_summer = go.Scatter(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer_predict, trace_fall_predict, trace_spring_predict, trace_summer, trace_fall, trace_spring]

    layout = dict(title = 'АРС',
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)

Лето: [-6601.609760517405, 2679.348317074274, -80.28844709468353]
Осень: [130587.64695749634, -20707.049569317307, 845.4800937212995]
Весна: [-35843.68202681324, 7798.725557080751, -249.17749407981694]


Лето: [32393.12591786536, -149.54229883083906, -6.777264124128048]
Осень: [13739.66936840018, 1669.461683867799, -36.467699659420035]
Весна: [15924.510200987694, 1266.5646410757558, -30.629596136917925]


Лето: [80099.09147759381, -6774.942372546716, 180.08591940169865]
Осень: [34032.78063137028, -2049.3930138667174, 67.42449018434786]
Весна: [-43955.90301326664, 7739.513988158118, -212.87507404573068]


Лето: [10734.528608485358, 5707.664768288073, -213.86575053858044]
Осень: [28468.67389126541, 506.7006896573552, 25.27944339015326]
Весна: [38163.240787815346, -2825.112577022555, 188.08191047653156]


Лето: [41730.0, 0.0, 0.0]
Осень: [16797.970463543446, 1969.8003718259447, -122.15296433836146]
Весна: [30385.53740008395, -3205.5991999056455, 160.16971583683804]


...

Лето: [-123898.01557653793, 26869.907958658812, -913.710673307207]
Осень: [14923.33438173437, 818.473896003113, -54.16605071835151]
Весна: [31061.194097686945, -172.555210069784, -26.361809360862704]


Лето: [3483.4071890280684, -1560.8321324033072, 227.4358261697548]
Осень: [31825.74838196772, -1556.424176014905, 36.827068659892376]
Весна: [9383.286110764437, 471.98092527420863, 10.45916832655983]


Лето: [55667.79313480946, -3514.3277437276734, 70.72844686670219]
Осень: [17175.0, 0.0, 0.0]
Весна: [24030.748874670597, -48.60122511216425, -5.249228148180922]


Лето: [9975.320205572114, 228.606599039446, 15.470400585104104]
Осень: [17812.920172913786, -789.0277025738718, 46.86735795510967]
Весна: [37369.61941016198, -4387.9425220711355, 217.96737241470478]


Лето: [43131.97390205103, -4332.850318908684, 221.95265396882675]
Осень: [61395.72242410036, -6068.977916168571, 227.02049999559065]
Весна: [-4870.211905456625, 3549.6545604187, -111.83793077768532]


In [260]:
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    ridge1 = Ridge()
    model1 = make_pipeline(PolynomialFeatures(2), ridge1)
    model1.fit(np.array(df_tmp.loc[df_tmp.summer == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean))

    trace_summer_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model1.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для лета',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )
    
    print('Лето: ' + str([ridge1.intercept_, ridge1.coef_[1], ridge1.coef_[2]]))

    ridge2 = Ridge()
    model2 = make_pipeline(PolynomialFeatures(2), ridge2)
    model2.fit(np.array(df_tmp.loc[df_tmp.fall == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean))

    trace_fall_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model2.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для осени',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Осень: ' + str([ridge2.intercept_, ridge2.coef_[1], ridge2.coef_[2]]))

    ridge3 = Ridge()
    model3 = make_pipeline(PolynomialFeatures(2), ridge3)
    model3.fit(np.array(df_tmp.loc[df_tmp.spring == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean))

    trace_spring_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model3.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для весны',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Весна: ' + str([ridge3.intercept_, ridge3.coef_[1], ridge3.coef_[2]]))

    trace_summer = go.Scatter(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer_predict, trace_fall_predict, trace_spring_predict, trace_summer, trace_fall, trace_spring]

    layout = dict(title = 'АРС',
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)

Лето: [4912.892987826373, -2499.0247395995675, 48.466702542948816]
Осень: [35073.43299763462, -9202.014465896023, 336.65990394027483]
Весна: [-34347.18168050263, 2731.9786771303297, -90.09069042312544]


Лето: [-15326.98640410964, 484.611501288194, -14.438308360112122]
Осень: [-14396.247822635196, 363.8858260774384, -7.502604758395109]
Весна: [0.0, 0.0, 0.0]


Лето: [23799.936833844826, -4669.810211394733, 138.59065029305347]
Осень: [-30226.171679982377, 2001.2863708610757, -71.39107433396948]
Весна: [-25446.51855034114, 550.7788446836986, 3.101366534712599]


Лето: [-45722.91578908308, 2788.1750689005803, -73.38732742314268]
Осень: [-18771.694122287765, -374.41650079535316, 15.572993140530674]
Весна: [0.0, 0.0, 0.0]


Лето: [-16419.0, 0.0, 0.0]
Осень: [-10486.655090960789, 1239.199588993671, -93.36160063503536]
Весна: [-41228.70520121686, 4555.382969977389, -181.90705075558054]


...

Лето: [18766.10562594002, -4082.526895911713, 81.3343575022134]
Осень: [1241.9250661403883, -165.4614419392376, -7.033883261724061]
Весна: [0.0, 0.0, 0.0]


Лето: [-61451.33106761599, 7081.690783543187, -264.85703845594645]
Осень: [7243.763219719654, -2829.874491677906, 100.82323282429059]
Весна: [-1876.5121718661885, 1436.1157120534633, -193.88811440871794]


Лето: [199222.2854890788, -36946.40755666605, 1283.9995662175]
Осень: [-7817.0, 0.0, 0.0]
Весна: [0.0, 0.0, 0.0]


Лето: [-37515.85539499225, 4479.647410591844, -165.17434666682254]
Осень: [22187.167228564686, -3527.053614630034, 87.72883492905886]
Весна: [28653.186057748484, -4689.86083719761, 124.00603677490145]


Лето: [3804.4471096844936, -2163.420577326104, 79.45570910524052]
Осень: [-18395.41325707539, 65.52138492156033, 1.5515516042440247]
Весна: [0.0, 0.0, 0.0]


In [262]:
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    ridge1 = Ridge()
    model1 = make_pipeline(PolynomialFeatures(2), ridge1)
    model1.fit(np.array(df_tmp.loc[df_tmp.summer == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean))

    trace_summer_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model1.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для лета',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )
    
    print('Лето: ' + str([ridge1.intercept_, ridge1.coef_[1], ridge1.coef_[2]]))

    ridge2 = Ridge()
    model2 = make_pipeline(PolynomialFeatures(2), ridge2)
    model2.fit(np.array(df_tmp.loc[df_tmp.fall == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean))

    trace_fall_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model2.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для осени',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Осень: ' + str([ridge2.intercept_, ridge2.coef_[1], ridge2.coef_[2]]))

    ridge3 = Ridge()
    model3 = make_pipeline(PolynomialFeatures(2), ridge3)
    model3.fit(np.array(df_tmp.loc[df_tmp.spring == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean))

    trace_spring_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model3.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для весны',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Весна: ' + str([ridge3.intercept_, ridge3.coef_[1], ridge3.coef_[2]]))

    trace_summer = go.Scatter(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer_predict, trace_fall_predict, trace_spring_predict, trace_summer, trace_fall, trace_spring]

    layout = dict(title = 'АРС',
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)

Лето: [24121.81276434438, 2513.00488679117, -57.977391638927095]
Осень: [25343.601466549902, 1939.9518017741848, -43.42480617221466]
Весна: [46034.89317682453, -1353.2479712602476, 51.877942177922435]


Лето: [65033.67484213076, -2271.492828258235, 58.26618709090499]
Осень: [14604.66639668474, 3018.699512231427, -50.67886012043027]
Весна: [23645.731581332242, 1015.4419326032379, 9.056233064024326]


Лето: [32717.940619057346, 1862.776166344609, -47.22336067750049]
Осень: [62648.369756892454, -2351.6931745064567, 89.72515200946799]
Весна: [36918.63924525945, 125.78049662325155, 16.45685521834374]


Лето: [63454.17061752858, -1069.5403615596344, 55.851169137539905]
Осень: [53629.95577872357, -536.8806118552845, 70.30849312028073]
Весна: [33835.1892251454, -332.1288039341633, 55.84959654543449]


Лето: [24065.0, 0.0, 0.0]
Осень: [58561.85345811257, -798.2254923875473, 68.35155458378463]
Весна: [87344.99356306306, -5036.791192801087, 156.6826592216469]


...

Лето: [141024.49610805212, -1233.7391338687075, -13.963528606881777]
Осень: [73697.66641685476, 7667.0021622146505, -352.0315702121719]
Весна: [123522.47338596918, -517.3154508868427, -129.3456183953917]


Лето: [46307.06482546685, 696.7223687431663, -63.665088611470864]
Осень: [159129.90987676522, -30964.71904250238, 1629.3534107104726]
Весна: [2784.437056078521, -248.48685966852287, 12.533174743095845]


Лето: [97108.91558581425, -4879.593663570887, 140.57273427616505]
Осень: [51165.0, 0.0, 0.0]
Весна: [56871.810753410915, -18.816947526547036, -4.6927891450883354]


Лето: [47964.560711644386, -6642.607317464372, 272.0239072206901]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [263]:
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    ridge1 = Ridge()
    model1 = make_pipeline(PolynomialFeatures(2), ridge1)
    model1.fit(np.array(df_tmp.loc[df_tmp.summer == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean))

    trace_summer_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model1.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для лета',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )
    
    print('Лето: ' + str([ridge1.intercept_, ridge1.coef_[1], ridge1.coef_[2]]))

    ridge2 = Ridge()
    model2 = make_pipeline(PolynomialFeatures(2), ridge2)
    model2.fit(np.array(df_tmp.loc[df_tmp.fall == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean))

    trace_fall_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model2.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для осени',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Осень: ' + str([ridge2.intercept_, ridge2.coef_[1], ridge2.coef_[2]]))

    ridge3 = Ridge()
    model3 = make_pipeline(PolynomialFeatures(2), ridge3)
    model3.fit(np.array(df_tmp.loc[df_tmp.spring == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean))

    trace_spring_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model3.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для весны',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Весна: ' + str([ridge3.intercept_, ridge3.coef_[1], ridge3.coef_[2]]))

    trace_summer = go.Scatter(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer_predict, trace_fall_predict, trace_spring_predict, trace_summer, trace_fall, trace_spring]

    layout = dict(title = 'АРС',
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)

Лето: [-144591.71608705405, 17047.10587235518, -308.4245026770121]
Осень: [83083.4496598258, -5697.389143994905, 249.87310868574764]
Весна: [53297.76922653188, -2029.8031963808885, 77.10490067478825]


Лето: [40286.181966694305, -375.87473173727574, 12.192962037236283]
Осень: [41100.201658511025, -844.1636578409671, 19.615323394931867]
Весна: [19233.226827473867, 1055.2277138867264, -25.721793034461633]


Лето: [47823.9863188043, 2452.556312984728, -50.21424479751397]
Осень: [72182.03334837632, -1112.5301574763973, 36.902891383767965]
Весна: [46916.964366955624, -690.186221042125, 47.08079212400599]


Лето: [43461.85956798544, -2675.0896175199755, 208.89824058636427]
Осень: [30672.889601362916, 769.7973483826208, -37.65177672496403]
Весна: [28787.66663714397, 465.8867895389904, -16.665432140777337]


Лето: [38230.0, 0.0, 0.0]
Осень: [78868.74197180306, -625.3144378166724, 60.30132349516153]
Весна: [100914.50558800336, -5919.408622142185, 195.25627335852514]


...

Лето: [35999.89166574295, 45.06543655363684, 164.53271037880452]
Осень: [87907.0645302327, -6285.338265370719, 238.7744275850999]
Весна: [43275.2655099924, 313.2957187378336, 70.81772021326093]


Лето: [61885.967060243056, -2327.8570301571444, 62.60642474030151]
Осень: [29078.43051776802, 4727.010790962866, -147.00971067060985]
Весна: [41227.17678640447, 3409.5062470052544, -138.96130701438747]


Лето: [7917.1755436907915, 4884.050556990451, -174.52142415057847]
Осень: [38145.0, 0.0, 0.0]
Весна: [35648.180650215494, -124.99466770291549, 26.95274550769729]


Лето: [44459.538934049124, -5800.778742977151, 261.5255994869381]
Осень: [96744.47623039228, -4830.42142156939, 205.57002213430692]
Весна: [39443.16218869117, 4927.460959599876, -193.09184761135714]


Лето: [41351.93162651907, -2751.7340835654168, 151.06234621545792]
Осень: [33187.87540047693, 451.05510608613116, 30.116169491014073]
Весна: [49820.15973254282, -19.70725809044374, -3.142730483991653]
